In [1]:
pip install git+https://github.com/RobustBench/robustbench.git

  Cloning https://github.com/RobustBench/robustbench.git to /tmp/pip-req-build-ta53jlkl
  Running command git clone --filter=blob:none --quiet https://github.com/RobustBench/robustbench.git /tmp/pip-req-build-ta53jlkl
  Resolved https://github.com/RobustBench/robustbench.git to commit 776bc95bb4167827fb102a32ac5aea62e46cfaab
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/fra31/auto-attack.git (to revision a39220048b3c9f2cca9a4d3a54604793c68eca7e) to /tmp/pip-install-ysy5eef0/autoattack_a93f3ee78d4d4416858af374103bd959
  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack.git /tmp/pip-install-ysy5eef0/autoattack_a93f3ee78d4d4416858af374103bd959
  Running command git rev-parse -q --verify 'sha^a39220048b3c9f2cca9a4d3a54604793c68eca7e'
  Running command git fetch -q https://github.com/fra31/auto-attack.git a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Resolved https://github.com/fra31/auto-attack.git to commit a39220048b3c9f2cca9a4

In [2]:
from robustbench.data import load_cifar10
from robustbench import load_model

x_test, y_test = load_cifar10(n_examples=50)

names = ['Bartoldson2024Adversarial_WRN-94-16','Amini2024MeanSparse','Bartoldson2024Adversarial_WRN-82-8','Bai2024MixedNUTS','Wang2023Better_WRN-70-16']
i=0
models=[0,0,0,0,0]
for model_name in names:
  models[i] = load_model(model_name=model_name, dataset='cifar10', threat_model='Linf')
  i= i+1


100%|██████████| 170498071/170498071 [00:13<00:00, 12932077.10it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


FileURLRetrievalError: Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1g6o9H1b6vjoBi1USdCBt64C8B8LPiioX

but Gdown can't. Please check connections and permissions.

In [ ]:
import torch
!pip install foolbox
import foolbox as fb
import numpy as np
from collections import defaultdict

In [ ]:

# fmodel es el modelo en Foolbox
# x_test es el tensor de datos de prueba (imágenes)
# y_test son las etiquetas de los datos de prueba

# Definir el ataque adversario (LinfPGD)
attack = fb.attacks.LinfPGD()

# Inicializar contadores para robustez por clase
class_correct_clean = defaultdict(int)  # Número de clasificaciones correctas en imágenes limpias
class_total = defaultdict(int)          # Número total de imágenes por clase
class_correct_adv = defaultdict(int)    # Número de clasificaciones correctas en imágenes adversarias

fmodel = fb.PyTorchModel(models[0], bounds=(0, 1))

# Realizar el ataque adversario con un epsilon de 8/255
_, advs, success = attack(fmodel, x_test.to('cuda:0'), y_test.to('cuda:0'), epsilons=[8/255])

# Convertir el tensor de éxito (success) a tipo float
success = success.float()

# Para cada imagen de prueba, verificar si la predicción es correcta en el set limpio y adversario
for i in range(len(y_test)):
    label = y_test[i].item()  # Obtener la etiqueta verdadera de la imagen i

    # Contar el número total de imágenes por clase
    class_total[label] += 1

    # Verificar si la predicción en la imagen limpia es correcta
    with torch.no_grad():
        pred_clean = torch.argmax(fmodel(x_test[i:i+1].to('cuda:0'))).item()
    if pred_clean == label:
        class_correct_clean[label] += 1

    # Verificar si la predicción en la imagen adversaria es correcta
    if not success[0][i]:  # success[0][i] indica si el ataque tuvo éxito (True significa que el ataque tuvo éxito)
        class_correct_adv[label] += 1

# Calcular y mostrar la robustez por clase
for cls in range(10):  # Para CIFAR-10 (ajustar para otros conjuntos de datos)
    clean_accuracy = class_correct_clean[cls] / class_total[cls]
    robust_accuracy = class_correct_adv[cls] / class_total[cls]
    print(f"Clase {cls}: Precisión limpia = {clean_accuracy:.2%}, Precisión robusta = {robust_accuracy:.2%}")


In [ ]:
# Definir el ataque adversario (LinfPGD)
attack = fb.attacks.LinfPGD()

# Inicializar contadores para robustez por clase
class_correct_clean = defaultdict(int)  # Número de clasificaciones correctas en imágenes limpias
class_total = defaultdict(int)          # Número total de imágenes por clase
class_correct_adv = defaultdict(int)    # Número de clasificaciones correctas en imágenes adversarias

fmodel = fb.PyTorchModel(models[1], bounds=(0, 1))

# Realizar el ataque adversario con un epsilon de 8/255
_, advs, success = attack(fmodel, x_test.to('cuda:0'), y_test.to('cuda:0'), epsilons=[8/255])

# Convertir el tensor de éxito (success) a tipo float
success = success.float()

# Para cada imagen de prueba, verificar si la predicción es correcta en el set limpio y adversario
for i in range(len(y_test)):
    label = y_test[i].item()  # Obtener la etiqueta verdadera de la imagen i

    # Contar el número total de imágenes por clase
    class_total[label] += 1

    # Verificar si la predicción en la imagen limpia es correcta
    with torch.no_grad():
        pred_clean = torch.argmax(fmodel(x_test[i:i+1].to('cuda:0'))).item()
    if pred_clean == label:
        class_correct_clean[label] += 1

    # Verificar si la predicción en la imagen adversaria es correcta
    if not success[0][i]:  # success[0][i] indica si el ataque tuvo éxito (True significa que el ataque tuvo éxito)
        class_correct_adv[label] += 1

# Calcular y mostrar la robustez por clase
for cls in range(10):  # Para CIFAR-10 (ajustar para otros conjuntos de datos)
    clean_accuracy = class_correct_clean[cls] / class_total[cls]
    robust_accuracy = class_correct_adv[cls] / class_total[cls]
    print(f"Clase {cls}: Precisión limpia = {clean_accuracy:.2%}, Precisión robusta = {robust_accuracy:.2%}")


Clase 0: Precisión limpia = 100.00%, Precisión robusta = 100.00%
Clase 1: Precisión limpia = 66.67%, Precisión robusta = 33.33%
Clase 2: Precisión limpia = 100.00%, Precisión robusta = 50.00%
Clase 3: Precisión limpia = 100.00%, Precisión robusta = 100.00%
Clase 4: Precisión limpia = 100.00%, Precisión robusta = 40.00%
Clase 5: Precisión limpia = 85.71%, Precisión robusta = 57.14%
Clase 6: Precisión limpia = 100.00%, Precisión robusta = 77.78%
Clase 7: Precisión limpia = 100.00%, Precisión robusta = 75.00%
Clase 8: Precisión limpia = 100.00%, Precisión robusta = 100.00%
Clase 9: Precisión limpia = 100.00%, Precisión robusta = 87.50%


In [ ]:
# Definir el ataque adversario (LinfPGD)
attack = fb.attacks.LinfPGD()

# Inicializar contadores para robustez por clase
class_correct_clean = defaultdict(int)  # Número de clasificaciones correctas en imágenes limpias
class_total = defaultdict(int)          # Número total de imágenes por clase
class_correct_adv = defaultdict(int)    # Número de clasificaciones correctas en imágenes adversarias

fmodel = fb.PyTorchModel(models[2], bounds=(0, 1))

# Realizar el ataque adversario con un epsilon de 8/255
_, advs, success = attack(fmodel, x_test.to('cuda:0'), y_test.to('cuda:0'), epsilons=[8/255])

# Convertir el tensor de éxito (success) a tipo float
success = success.float()

# Para cada imagen de prueba, verificar si la predicción es correcta en el set limpio y adversario
for i in range(len(y_test)):
    label = y_test[i].item()  # Obtener la etiqueta verdadera de la imagen i

    # Contar el número total de imágenes por clase
    class_total[label] += 1

    # Verificar si la predicción en la imagen limpia es correcta
    with torch.no_grad():
        pred_clean = torch.argmax(fmodel(x_test[i:i+1].to('cuda:0'))).item()
    if pred_clean == label:
        class_correct_clean[label] += 1

    # Verificar si la predicción en la imagen adversaria es correcta
    if not success[0][i]:  # success[0][i] indica si el ataque tuvo éxito (True significa que el ataque tuvo éxito)
        class_correct_adv[label] += 1

# Calcular y mostrar la robustez por clase
for cls in range(10):  # Para CIFAR-10 (ajustar para otros conjuntos de datos)
    clean_accuracy = class_correct_clean[cls] / class_total[cls]
    robust_accuracy = class_correct_adv[cls] / class_total[cls]
    print(f"Clase {cls}: Precisión limpia = {clean_accuracy:.2%}, Precisión robusta = {robust_accuracy:.2%}")


Clase 0: Precisión limpia = 100.00%, Precisión robusta = 100.00%
Clase 1: Precisión limpia = 66.67%, Precisión robusta = 33.33%
Clase 2: Precisión limpia = 100.00%, Precisión robusta = 100.00%
Clase 3: Precisión limpia = 100.00%, Precisión robusta = 100.00%
Clase 4: Precisión limpia = 100.00%, Precisión robusta = 60.00%
Clase 5: Precisión limpia = 85.71%, Precisión robusta = 57.14%
Clase 6: Precisión limpia = 100.00%, Precisión robusta = 77.78%
Clase 7: Precisión limpia = 100.00%, Precisión robusta = 75.00%
Clase 8: Precisión limpia = 100.00%, Precisión robusta = 100.00%
Clase 9: Precisión limpia = 100.00%, Precisión robusta = 87.50%


In [ ]:
# Definir el ataque adversario (LinfPGD)
attack = fb.attacks.LinfPGD()

# Inicializar contadores para robustez por clase
class_correct_clean = defaultdict(int)  # Número de clasificaciones correctas en imágenes limpias
class_total = defaultdict(int)          # Número total de imágenes por clase
class_correct_adv = defaultdict(int)    # Número de clasificaciones correctas en imágenes adversarias

fmodel = fb.PyTorchModel(models[3], bounds=(0, 1))

# Realizar el ataque adversario con un epsilon de 8/255
_, advs, success = attack(fmodel, x_test.to('cuda:0'), y_test.to('cuda:0'), epsilons=[8/255])

# Convertir el tensor de éxito (success) a tipo float
success = success.float()

# Para cada imagen de prueba, verificar si la predicción es correcta en el set limpio y adversario
for i in range(len(y_test)):
    label = y_test[i].item()  # Obtener la etiqueta verdadera de la imagen i

    # Contar el número total de imágenes por clase
    class_total[label] += 1

    # Verificar si la predicción en la imagen limpia es correcta
    with torch.no_grad():
        pred_clean = torch.argmax(fmodel(x_test[i:i+1].to('cuda:0'))).item()
    if pred_clean == label:
        class_correct_clean[label] += 1

    # Verificar si la predicción en la imagen adversaria es correcta
    if not success[0][i]:  # success[0][i] indica si el ataque tuvo éxito (True significa que el ataque tuvo éxito)
        class_correct_adv[label] += 1

# Calcular y mostrar la robustez por clase
for cls in range(10):  # Para CIFAR-10 (ajustar para otros conjuntos de datos)
    clean_accuracy = class_correct_clean[cls] / class_total[cls]
    robust_accuracy = class_correct_adv[cls] / class_total[cls]
    print(f"Clase {cls}: Precisión limpia = {clean_accuracy:.2%}, Precisión robusta = {robust_accuracy:.2%}")


/usr/local/lib/python3.10/dist-packages/robustbench/model_zoo/architectures/mixing_net.py:237: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=enable_autocast):


Clase 0: Precisión limpia = 100.00%, Precisión robusta = 100.00%
Clase 1: Precisión limpia = 66.67%, Precisión robusta = 33.33%
Clase 2: Precisión limpia = 100.00%, Precisión robusta = 100.00%
Clase 3: Precisión limpia = 100.00%, Precisión robusta = 100.00%
Clase 4: Precisión limpia = 100.00%, Precisión robusta = 60.00%
Clase 5: Precisión limpia = 85.71%, Precisión robusta = 57.14%
Clase 6: Precisión limpia = 100.00%, Precisión robusta = 88.89%
Clase 7: Precisión limpia = 100.00%, Precisión robusta = 75.00%
Clase 8: Precisión limpia = 100.00%, Precisión robusta = 100.00%
Clase 9: Precisión limpia = 100.00%, Precisión robusta = 87.50%


In [ ]:
# Definir el ataque adversario (LinfPGD)
attack = fb.attacks.LinfPGD()

# Inicializar contadores para robustez por clase
class_correct_clean = defaultdict(int)  # Número de clasificaciones correctas en imágenes limpias
class_total = defaultdict(int)          # Número total de imágenes por clase
class_correct_adv = defaultdict(int)    # Número de clasificaciones correctas en imágenes adversarias

fmodel = fb.PyTorchModel(models[4], bounds=(0, 1))

# Realizar el ataque adversario con un epsilon de 8/255
_, advs, success = attack(fmodel, x_test.to('cuda:0'), y_test.to('cuda:0'), epsilons=[8/255])

# Convertir el tensor de éxito (success) a tipo float
success = success.float()

# Para cada imagen de prueba, verificar si la predicción es correcta en el set limpio y adversario
for i in range(len(y_test)):
    label = y_test[i].item()  # Obtener la etiqueta verdadera de la imagen i

    # Contar el número total de imágenes por clase
    class_total[label] += 1

    # Verificar si la predicción en la imagen limpia es correcta
    with torch.no_grad():
        pred_clean = torch.argmax(fmodel(x_test[i:i+1].to('cuda:0'))).item()
    if pred_clean == label:
        class_correct_clean[label] += 1

    # Verificar si la predicción en la imagen adversaria es correcta
    if not success[0][i]:  # success[0][i] indica si el ataque tuvo éxito (True significa que el ataque tuvo éxito)
        class_correct_adv[label] += 1

# Calcular y mostrar la robustez por clase
for cls in range(10):  # Para CIFAR-10 (ajustar para otros conjuntos de datos)
    clean_accuracy = class_correct_clean[cls] / class_total[cls]
    robust_accuracy = class_correct_adv[cls] / class_total[cls]
    print(f"Clase {cls}: Precisión limpia = {clean_accuracy:.2%}, Precisión robusta = {robust_accuracy:.2%}")


Clase 0: Precisión limpia = 100.00%, Precisión robusta = 100.00%
Clase 1: Precisión limpia = 100.00%, Precisión robusta = 33.33%
Clase 2: Precisión limpia = 100.00%, Precisión robusta = 50.00%
Clase 3: Precisión limpia = 100.00%, Precisión robusta = 100.00%
Clase 4: Precisión limpia = 100.00%, Precisión robusta = 60.00%
Clase 5: Precisión limpia = 85.71%, Precisión robusta = 57.14%
Clase 6: Precisión limpia = 100.00%, Precisión robusta = 77.78%
Clase 7: Precisión limpia = 100.00%, Precisión robusta = 75.00%
Clase 8: Precisión limpia = 100.00%, Precisión robusta = 100.00%
Clase 9: Precisión limpia = 100.00%, Precisión robusta = 87.50%


In [ ]:
i=0
for model in models:
  total_params = sum(p.numel() for p in model.parameters())
  print(f'El modelo {i} tiene {total_params} parámetros.')
  i+=1

El modelo 0 tiene 365915610 parámetros.
El modelo 1 tiene 267240802 parámetros.
El modelo 2 tiene 79125466 parámetros.
El modelo 3 tiene 499519993 parámetros.
El modelo 4 tiene 266796506 parámetros.
